# MLOps

parquet file ->(raw data) -> Data Preparation ->(features),(targets) -> Model Training -> (artifact) -> ML model

# Feature Store

hopswork.ai

generate API key to use the Python SDK to communicate with the feature store

In [62]:
from src.config import PROJECT_NAME, API_KEY
from src.paths import TIME_SERIES_DATA_DIR


In [75]:
import hopsworks

project = hopsworks.login(project=PROJECT_NAME, api_key_value=API_KEY)
feature_store = project.get_feature_store()
FEATURE_GROUP_NAME = 'time_series_hourly_feature_group'
FEATURE_GROUP_VERSION = 1

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1152153
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
from src.paths import TRANSFORMED_DATA_DIR
import pandas as pd
data = pd.read_parquet(TRANSFORMED_DATA_DIR / 'yellow_tripdata_features_target.parquet')

In [6]:
data

,rides_previous_672,rides_previous_671,rides_previous_670,rides_previous_669,rides_previous_668,rides_previous_667,rides_previous_666,rides_previous_665,rides_previous_664,rides_previous_663,...,rides_previous_7,rides_previous_6,rides_previous_5,rides_previous_4,rides_previous_3,rides_previous_2,rides_previous_1,pickup_hour,PULocationID,target_rides_next_hour
0,11.0,15.0,26.0,8.0,9.0,7.0,3.0,1.0,0.0,3.0,...,11.0,7.0,4.0,3.0,4.0,9.0,19.0,2022-01-29 00:00:00,4,15.0
1,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,...,5.0,5.0,4.0,10.0,7.0,5.0,9.0,2022-01-29 23:00:00,4,9.0
2,0.0,3.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,3.0,...,13.0,6.0,8.0,7.0,8.0,5.0,5.0,2022-01-30 22:00:00,4,0.0
3,7.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,3.0,2.0,...,2.0,3.0,6.0,3.0,16.0,7.0,1.0,2022-01-31 21:00:00,4,1.0
4,3.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,4.0,2.0,5.0,3.0,8.0,3.0,2022-02-01 20:00:00,4,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259839,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-08-28 01:00:00,110,0.0
259840,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-08-29 00:00:00,110,0.0
259841,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-08-29 23:00:00,110,0.0
259842,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-08-30 22:00:00,110,0.0


In [69]:
ts_data = pd.read_parquet(TIME_SERIES_DATA_DIR / 'yellow_tripdata_time_series.parquet')

In [70]:
ts_data

,pickup_hour,rides,PULocationID
0,2022-01-01 00:00:00,11.0,4
1,2022-01-01 01:00:00,15.0,4
2,2022-01-01 02:00:00,26.0,4
3,2022-01-01 03:00:00,8.0,4
4,2022-01-01 04:00:00,9.0,4
...,...,...,...
6147883,2024-08-31 19:00:00,0.0,110
6147884,2024-08-31 20:00:00,0.0,110
6147885,2024-08-31 21:00:00,0.0,110
6147886,2024-08-31 22:00:00,0.0,110


In [78]:
# string to datetime
ts_data['pickup_hour'] = pd.to_datetime(ts_data['pickup_hour'], utc=True)

# add column with Unix epoch milliseconds
ts_data['pickup_ts'] = ts_data['pickup_hour'].astype(int) // 10**6
# Renomear a coluna no DataFrame
ts_data = ts_data.rename(columns={'PULocationID': 'pickup_location_id'})


In [79]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_ts'],
    event_time='pickup_ts',
)
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1152153/fs/1142856/fg/1332571


Uploading Dataframe: 0.00% |          | Rows 0/6147888 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1152153/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7efc8a24ca30>, None)